In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import pathlib
import os
import time

In [ ]:
metadataColNames = ["Algorithm", "Version", "Timestamp", "Satellite", "Instrument", "FlightModel", "ScanMode", "ProductType", "FileName", "DataSource",
               "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "MissingValueCode"]

dataColNames = ['Latitude', 'Longitude', 'Code', 'FRP', 'Fire Size', 'Fire Temp', 'Line', 'Element', 'Pixel Size',  'Obs BT4', 'Obs BT11', 'Bkg BT4', 'Bkg BT11', 'SolZen',  'SatZen', 'RelAzi', 'Eco']

allColNames = metadataColNames + dataColNames


In [ ]:
starttime = time.time()
wfabba_dates = ["GOES-16-2019", "GOES-17-2019", "GOES-16-2020", "GOES-17-2020", "GOES-16-2021", "GOES-17-2021", "GOES-16-Jan-2021","GOES-17-Jan-2021"]

rawDir = "../../data/raw/wfabba/"

for wfabba_date in wfabba_dates:
    masterArr = []
    print(wfabba_date)

    targetDir = "../../data/raw/wfabba/" + wfabba_date

    for root, subdirs, files in sorted(os.walk(targetDir)):
        print(root)
        print(len(masterArr))

        if len(files) > 0:
            # if files[0][-4:] == ".txt":
            if files[0][-8:] == ".GOES-16" or files[0][-8:] == ".GOES-17" or files[0][-12:] == ".GOES-16.txt" or files[0][-12:] == ".GOES-17.txt" :
                # print(root + "/"+files[0])
                for file in files:
                    filepath = root+"/"+file
                    if files[0][-8:] != ".GOES-16" and files[0][-8:] == ".GOES-17" and files[0][-12:] == ".GOES-16.txt" and files[0][-12:] == ".GOES-17.txt" :
                        print(filepath)
                        continue
                    fo = open(filepath ,'r')
                    metadataDict = {}
                    dataArr = []
                    lineCount = 0
                    try:
                        for line in fo:
        #                     print(line)
                            if line[0:4] == "### ":
                                if lineCount < 11:
                                    parseString = line[4:]
                                    parseString = parseString.strip()
        #                             print("Parse String:",parseString)

                                    if parseString[0:4] == 'Date':
                                        if parseString == "Date: , Time:  UTC":
                                            break
                                        year = parseString[6:10]
                                        day_num = parseString[10:13]
                                        hr = parseString[21:23]
                                        min = parseString[24:26]
                                        sec = parseString[27:29]
                                        
                                        if year == "0000" and day_num == "000" and hr == "00" and min == "00" and sec == "00":
                                            break
                                            
                                        res = dt.datetime.strptime(year + "-" + day_num + " " + hr + ":" + min + ":" + sec, "%Y-%j %H:%M:%S")
                                        metadataDict["Timestamp"] = res
                                    else:
                                        splitData = parseString.split(", ")
                                        for item in splitData:
                                            if item[-9:] == 'Algorithm':
                                                metadataDict["Algorithm"] = item
                                            else:
                                                sections = item.split(":")
                                                # print(sections)

                                                if sections[0] == 'Flight Model':
                                                    keyname = 'FlightModel'
                                                elif sections[0] == 'Scan Mode':
                                                    keyname = 'ScanMode'
                                                elif sections[0] == 'Product type':
                                                    keyname = 'ProductType'
                                                elif sections[0] == 'Product/L2 filename':
                                                    keyname = 'FileName'
                                                elif sections[0] == 'Data source':
                                                    keyname = 'DataSource'
                                                elif sections[0] == 'Data creation time stamp':
                                                    keyname = 'DataCreationTimestamp'
                                                elif sections[0] == 'Navigation projection subpoint longitude':
                                                    keyname = 'NavProjSubPtLong'
                                                elif sections[0] == 'Actual satellite subpoint longitude':
                                                    keyname = 'ActualSatSubPtLong'
                                                elif sections[0] == 'Number of detected fires':
                                                    keyname = 'NumFire'
                                                elif sections[0] == 'Missing value code':
                                                    keyname = 'MissingValueCode'
                                                else:
                                                    keyname = sections[0]

                                                if len(sections) > 1:
                                                    if sections[1] == '':
                                                        value = None
                                                    else:
                                                        value = sections[1].strip()
                                                    metadataDict[keyname] = value
                                    lineCount+=1
                            elif line[0:3] != '###':
                                line_list = line.replace(' ', '').replace("\n","").split(',')
                                app_dict = {dataColNames[i]: line_list[i] for i in range(len(dataColNames))}

                                if (float(app_dict["Latitude"]) >= 32) and (float(app_dict["Latitude"]) <= 35) and (float(app_dict["Longitude"]) >= -123) and (float(app_dict["Longitude"]) <= -113):
                                    dataArr.append(app_dict)
                    except:
                        break
                        
                    for dataDict in dataArr:
                        allDataDict = {}
                        allDataDict.update(metadataDict)
                        allDataDict.update(dataDict)
                        masterArr.append(allDataDict)
                        
                    fo.close()

    df = pd.DataFrame(masterArr, columns=allColNames)
    endtime = time.time()
    print(endtime-starttime)
    processedDir = "../../data/processed/wfabba/"
    df.to_csv(processedDir + wfabba_date+".csv")
    print("==========================")